In [ ]:
# This notebook is deprecated; check 06_adversarial.ipynb

In [1]:
# hide
%load_ext autoreload
%autoreload 2
%matplotlib inline 

from ipynb_path import *

In [2]:
from counterfactual.import_essentials import *
from counterfactual.utils import *
from counterfactual.train import train_model
from counterfactual.evaluate import load_trained_model, model_cf_gen
from counterfactual.net import BaselineModel, CounterfactualModel
from counterfactual.training_module import BaseModule, CounterfactualTrainingModule
from pytorch_lightning.metrics.functional import accuracy
from sklearn.linear_model import LogisticRegression

pl_logger = logging.getLogger('lightning')

In [3]:
class A:
    a = 1
    b = 2
    def __init__(self) -> None:
        setattr(self, 'a', 3)

## Data

### Simulated

In [3]:
def generate_data(data_path: Path, means, stds):
    x_1_data = {
        f'x{i+1}': np.random.normal(-mean, std, 1000) for i, (mean, std) in enumerate(zip(means, stds))
    }
    x_1_data.update({'y': np.ones((1000,))})
    x_0_data = {
        f'x{i+1}': np.random.normal(mean, std, 1000) for i, (mean, std) in enumerate(zip(means, stds))
    }
    x_0_data.update({'y': np.zeros((1000,))})
    X = pd.concat((
        pd.DataFrame(x_1_data), pd.DataFrame(x_0_data)
    ))
    X = shuffle(X)
    X.to_csv(data_path, index=False)
    return X


def generate_shift_data(dimensions: int, data_dir: Path):
    data_dir = Path(data_dir)
    if not data_dir.is_dir():
        data_dir.mkdir()
    org_data = generate_data(data_dir / Path('original.csv'), 
        means=[2 for _ in range(dimensions)], stds=[0.5 for _ in range(dimensions)])
    upt_data = generate_data(data_dir / Path('updated.csv'), 
        means=[1. + (random.random() / 5) for _ in range(dimensions)], stds=[1. for _ in range(dimensions)])
    org_model = LogisticRegression().fit(org_data[[f'x{i+1}' for i in range(dimensions)]], org_data['y'])
    upt_model = LogisticRegression().fit(upt_data[[f'x{i+1}' for i in range(dimensions)]], upt_data['y'])
    print(f"org_model: {org_model.coef_}")
    print(f"upt_model: {upt_model.coef_}")
    
    

### Loan

In [6]:
!unzip assets/data/adv/loan.csv.zip 

Archive:  assets/data/adv/loan.csv.zip
  inflating: SBAnational.csv         


In [3]:
loan = pd.read_csv('assets/data/adv/loan.csv').drop(
    ['LoanNr_ChkDgt', 'Name', 'City', 'Zip', 'Bank', 'RevLineCr', 'BalanceGross',
    'ApprovalDate', 'ChgOffDate', 'DisbursementDate', 'BankState', 'FranchiseCode'], 
    axis=1)
loan = loan.infer_objects()

/home/birk/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
loan = loan.dropna()

In [5]:
# https://www.kaggle.com/mirbektoktogaraev/should-this-loan-be-approved-or-not
#Function extracts first 2 digits from variable, if variables = 0, returns 0
def first_two(d):
    if d <= 0:
        return 0
    return (d // 10 ** (int(math.log(d, 10)) - 1))

#Function returns points according to the given rate. Lower rate, higher points.
def point_def(rate):
    if rate <= 12:
        return 5
    elif  12 < rate <= 17: 
        return 4
    elif 17 < rate <= 21:
        return 3
    elif 21 < rate <= 25:
        return 2
    elif rate > 25:
        return 1

#Function returns points according to the default rate of each sector.
def apply_score(i):
    sector_default = {21 : 8, 11 : 9, 55 : 10, 62:10, 22:14, 92:15,54:19, 42:19,31:19,32:16,33:14,81:20,71:21,72:22,44:22,45:23,23:23,56:24,61:24,51:25,48:27,49:23,52:28,53:29}
    if i > 0:
        defrate = None
        if i in sector_default:
            defrate = sector_default[i]
            return point_def(defrate)
    return 0

loan['Sector_Points'] = loan['NAICS'].apply(first_two).apply(apply_score)
loan = loan.drop(['NAICS'], axis=1)


In [6]:
# https://www.kaggle.com/evgenyparenchenkov/random-forest-for-loan-classification
# Let's convert the strings styled as '$XXXX.XX' to float values
money_cols = ['ChgOffPrinGr','DisbursementGross', 'GrAppv', 'SBA_Appv']

for col in money_cols:
  loan[col] = [float(val[1:].replace(',', '')) for val in loan[col].values]


In [7]:
loan['DisbursementGross'] = np.log(loan['DisbursementGross'])
loan['GrAppv'] = np.log(loan['GrAppv'])
loan['SBA_Appv'] = np.log(loan['SBA_Appv'])
loan = loan.drop(['ChgOffPrinGr'], axis=1)

In [8]:
loan['Approve'] = (loan['MIS_Status'] == 'P I F').astype(np.int)
loan = loan.drop(['MIS_Status'], axis=1)

In [9]:
loan['ApprovalFY'] = loan['ApprovalFY'].apply(lambda x: 1976 if x == '1976A' else x)
loan['ApprovalFY'] = loan['ApprovalFY'].astype(np.int)

In [10]:
loan['Term'] = loan['Term'] // 12

In [11]:
loan_pos = loan[loan['Approve'] == 1]
loan_neg = loan[loan['Approve'] == 0]
loan_pos = loan_pos.sample(frac=0.5, random_state=0)

loan = pd.concat([loan_pos, loan_neg])

In [12]:
loan = loan.sort_values('ApprovalFY')

In [13]:
loan = loan[(loan['ApprovalFY'] > 1982) & (loan['ApprovalFY'] < 2014)]

In [14]:
loan.to_csv("assets/data/adv/loan/data.csv", index=None)

In [15]:
loan = pd.read_csv('assets/data/adv/loan/data.csv')

In [16]:
loan['ApprovalFY'].value_counts(ascending=True)

1983     1084
2013     1202
1984     1361
1985     1372
1988     1408
1986     1505
1987     1575
2012     3103
2011     6586
1989     6979
1990     7684
1991     8069
2010     9237
1992    10675
2009    11009
1993    11957
1994    16230
1998    19568
1997    19924
1999    20578
2000    20654
1996    20813
2001    20889
1995    23358
2002    24834
2008    27332
2003    33173
2004    40312
2005    48329
2007    51025
2006    51124
Name: ApprovalFY, dtype: int64

In [28]:
for year in range(1988, 2013):
    csv_file_name = f"assets/data/adv/loan/year={year}.csv"
    shuffle(loan[loan['ApprovalFY'] == year]).drop(['ApprovalFY'], axis=1)\
        .to_csv(csv_file_name, index=None)

In [20]:
size = len(loan) // 10
size

52549

In [21]:
for i in range(10):
    loan[size * i: size * (i+1)].to_csv(f'assets/data/adv/loan/w_{i}.csv', index=None)

#### Split Two 

In [4]:
data = pd.read_csv('assets/data/adv/loan/data.csv')

In [6]:
size = len(data) // 2
size

262747

In [7]:
data[:size].to_csv("assets/data/adv/loan/org.csv", index=None)
data[:size].to_csv("assets/data/adv/loan/upt.csv", index=None)

## Model

### Simulated

In [4]:
t_config = {
    "max_epochs": 150,
    "gpus": 1,
    "deterministic": True,
    "log_every_n_steps": 1,
    "automatic_optimization": False
}

In [8]:
def simulated_experiment(n_features: int):
    generate_shift_data(n_features, data_dir='assets/data/adv/simulated')
    config = {
        "data_dir": "assets/data/adv/simulated/original.csv",
        "data_name": "simulated",
        "lr": 0.003,
        "batch_size": 128,
        "lambda_1": 1.0,
        "lambda_2": 0.01,
        "lambda_3": 0.2,
        "threshold": 1.0,
        "continous_cols": [
            f'x{i + 1}' for i in range(n_features)
        ],
        "discret_cols": [],
        "encoder_dims": [
            n_features,
            100,
            10
        ],
        "decoder_dims": [
            10,
            10
        ],
        "explainer_dims": [
            10,
            10
        ],
        "loss_1": "mse",
        "loss_2": "mse",
        "loss_3": "mse",
        "adv": False,
        "perturb_target": "input",
        "perturb_latent_target": "z_p"
    }
    data_dir_list = [
        "assets/data/adv/simulated/original.csv",
        "assets/data/adv/simulated/updated.csv"
    ]
    no_adv_result = experiment_step(config, t_config, data_dir_list)
    config['adv'] = True
    adv_result = experiment_step(config, t_config, data_dir_list)
    return no_adv_result, adv_result



In [9]:
no_adv_results = []
adv_results = []

for n_features in [2, 5, 10, 20]:
    no_adv_result, adv_result = simulated_experiment(n_features)
    no_adv_results.append(no_adv_result)
    adv_results.append(adv_result)


total data length: 4000
preprocessing continuous features...
start training...
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Disable automatic optimization with the trainer flag is deprecated and will be removed in v1.3.0!Please use the property on the LightningModule for disabling automatic optimization
  warnings.warn(*args, **kwargs)
hyper parameters: "adv":                   False
"batch_size":            128
"continous_cols":        ['x1', 'x2']
"data_dir":              assets/data/adv/simulated/original.csv
"data_name":             simulated
"decoder_dims":          [10, 10]
"discret_cols":          []
"encoder_dims":          [2, 100, 10]
"explainer_dims":        [10, 10]
"lambda_1":              1.0
"lambda_2":              0.01
"lambda_3":              0.2
"loss_1":                mse
"loss_2":                mse

org_model: [[-2.06537047 -2.09513896]]
upt_model: [[-2.11484324 -1.89750492]]



  | Name      | Type                 | Params | In sizes | Out sizes
--------------------------------------------------------------------------
0 | model     | _CounterfactualModel | 1.7 K  | ?        | ?        
1 | model_dup | _CounterfactualModel | 1.7 K  | ?        | ?        
--------------------------------------------------------------------------
3.3 K     Trainable params
0         Non-trainable params
3.3 K     Total params
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [13]:
def avg_validity_matrix(results):
    matrix_all = [result['validity_matrix'] for result in results]
    return np.average(matrix_all, axis=0)

varience shift

In [16]:
avg_validity_matrix(no_adv_results[:4])

array([[1.   , 1.   ],
       [0.872, 1.   ]])

In [17]:
avg_validity_matrix(adv_results[:4])

array([[1.        , 1.        ],
       [0.99950001, 0.99950001]])

mean & varience shift

In [22]:
avg_validity_matrix(no_adv_results[:4])

array([[1.    , 1.    ],
       [0.874 , 0.9985]])

In [23]:
avg_validity_matrix(adv_results[:4])

array([[1.        , 1.        ],
       [0.7515    , 0.99950001]])

### Latent Vector

In [10]:
def latent_vis(model: CounterfactualModel):
    from sklearn.manifold import TSNE
    import seaborn as sns
    x, label = model.val_dataset[:]
    x_pos = x[label == 1.]
    x_neg = x[label == 0.]
    z_pos, pred_pos = model.latent_vec(x_pos)
    z_neg, pred_neg = model.latent_vec(x_neg)
    z = torch.cat((z_pos[:10000], z_neg[:10000]), dim=0).detach()
    pred = torch.cat((pred_pos[:10000], pred_neg[:10000]), dim=0).detach()
    y = torch.cat((torch.ones(10000), torch.zeros(10000)), dim=0)
    tsne = TSNE(n_iter=300, n_jobs=-1)
    tsne_z = tsne.fit_transform(z)
    tsne_pred = tsne.fit_transform(pred)
    sns.scatterplot(
        x=tsne_z[::10, 0], y=tsne_z[::10, 1],
        hue=y[::10]
    )
    sns.scatterplot(
        x=tsne_pred[::10, 0], y=tsne_pred[::10, 1],
        hue=y[::10]
    )
    

## Analysis

In [3]:
adv_false = pd.read_csv("assets/result/loan/result_adv=False_eps=auto_alpha=auto.csv", index_col=0)
adv_true = pd.read_csv("assets/result/loan/result_adv=True_eps=auto_alpha=auto.csv", index_col=0)


In [4]:
adv_false

,m_0,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9
m_0,0.995890,0.131299,0.603288,0.300274,0.250799,0.761227,0.846780,0.820901,0.099406,0.927386
m_1,0.018344,0.992845,0.145228,0.175978,0.401203,0.152306,0.018572,0.362079,0.992084,0.992160
m_2,0.071472,0.184807,0.992769,0.251408,0.561805,0.088979,0.094916,0.476709,0.656416,0.919851
m_3,0.631299,0.209469,0.313137,0.992008,0.743721,0.328208,0.302481,0.187243,0.154742,0.238012
m_4,0.280484,0.322195,0.332318,0.401812,0.992465,0.642183,0.233978,0.602908,0.779799,0.796849
m_5,0.519866,0.417415,0.634419,0.192876,0.428680,0.992693,0.680926,0.542853,0.795631,0.910336
m_6,0.969097,0.622089,0.588750,0.688842,0.377074,0.600700,0.993987,0.886512,0.817172,0.810017
m_7,0.701857,0.778048,0.720962,0.239458,0.542016,0.693408,0.881717,0.997945,0.932638,0.655960
m_8,0.750190,0.855457,0.682524,0.279799,0.479525,0.765870,0.745776,0.739839,0.995509,0.691658
m_9,0.292815,0.623078,0.519714,0.417872,0.656569,0.592860,0.407216,0.656949,0.666920,0.989877


In [5]:
adv_true

,m_0,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9
m_0,0.865124,0.291901,0.672781,0.303547,0.312148,0.716700,0.451895,0.546202,0.712666,0.249201
m_1,0.833080,0.980667,0.546887,0.959963,0.355838,0.735957,0.131299,0.974654,0.550236,0.027554
m_2,0.293956,0.109225,0.958898,0.109834,0.937662,0.720505,0.840463,0.973207,0.701020,0.177805
m_3,0.599939,0.859720,0.670650,0.846019,0.411021,0.604582,0.371974,0.849825,0.725225,0.360253
m_4,0.285051,0.342137,0.409804,0.217232,0.942914,0.346704,0.577713,0.365200,0.690135,0.275993
m_5,0.803699,0.518648,0.474273,0.362156,0.507764,0.977774,0.861775,0.782463,0.948851,0.345410
m_6,0.526260,0.471609,0.603212,0.418937,0.457908,0.466509,0.931192,0.621936,0.494900,0.340234
m_7,0.670802,0.762216,0.585782,0.781474,0.580530,0.896712,0.756051,0.971000,0.717689,0.909347
m_8,0.378596,0.762521,0.535926,0.743644,0.883392,0.799513,0.808342,0.851271,0.968641,0.801568
m_9,0.773710,0.779571,0.717232,0.298143,0.759096,0.701324,0.612194,0.899528,0.845106,0.869234


In [9]:
def avg_decrease(matrix):
    validity = [matrix[i][i] for i in range(10)]
    print(f"[cf_validity] avg={np.average(validity):.4f}; std={np.std(validity):.4f}")

    result = []
    for i in range(9):
        result += [matrix[i][i] - matrix[i][i+1]]
    print(f"[w=1]: avg=-{np.average(result):.4f}; std={np.std(result):.4f}")

    result =  []
    for i in range(10):
        for j in range(10):
            if j == i:
                continue
            result += [matrix[i][i] - matrix[i][j]]
    print(f"[all]: avg=-{np.average(result):.4f}; std={np.std(result):.4f}")

In [10]:
avg_decrease(adv_false.to_numpy())

[cf_validity] avg=0.9936; std=0.0022
[w=1]: avg=-0.4267; std=0.2918
[all]: avg=-0.4685; std=0.2645


In [11]:
avg_decrease(adv_true.to_numpy())

[cf_validity] avg=0.9311; std=0.0490
[w=1]: avg=-0.4148; std=0.2199
[all]: avg=-0.3443; std=0.2366


In [9]:
avg_decrease(pd.read_csv('assets/result/result_adv=True_eps=0.1_alpha=0.1.csv', 
    index_col=0))

[cf_validity] avg=0.9784; std=0.0144
[w=1]: avg=-0.3349; std=0.1781
[all]: avg=-0.3864; std=0.1579
